In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

import datetime
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math


In [2]:
df = pd.read_csv("../data/viewing_rating.csv", encoding='cp932')

In [3]:
df.start_at=pd.to_datetime(df.start_at)


In [4]:
df["drama_key"] = df.start_at.dt.strftime("%y")+df.start_at.dt.strftime("%m")+"_"+df.TV_station+"_"+df.time_table

In [5]:
df
# df.start_at.dt.strftime("%m")

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,...,8.8,9.8,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,...,9.7,-,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,...,9.6,-,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,...,13.7,-,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,...,19.3,-,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,...,-,-,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,...,3.2,-,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,...,9.2,-,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,...,-,-,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,...,10.4,-,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22


In [6]:
# import pandas_profiling as pdp
# pdp.ProfileReport(df)

In [7]:
import json

f = open("../data/drama_info.json", 'r')
print(f)
drama_info_json = json.load(f)

<_io.TextIOWrapper name='../data/drama_info.json' mode='r' encoding='UTF-8'>


In [8]:
# def addDramaInfo (drama_title) 
#     main_actor_only_json[drama_title]
    


In [9]:
print (drama_info_json)

{'1910_CX_月21': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_水22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_木23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_金22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_金23': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_土22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_土23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_日21': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_日22': {'main_actor1': '', 'main_actor2': ''}, '1907_CX_月21': {'main_actor1': '上野樹里', 'main_actor2': '時任三郎'}, '1907_CX_火21': {'main_actor1': '三浦春馬', 'main_actor2': '芳根京子'}, '1907_TBS_火22': {'main_actor1': '石原さとみ', 'main_actor2': '福士蒼汰'}, '1907_NTV_水22': {'main_actor1': '杏', 'main_actor2': '宮沢氷魚'}, '1907_EX_木21': {'main_act

In [10]:
df["main_actor1"] = ""
df["main_actor2"] = ""

In [11]:
for tmp_index in range(len(df)):
    tmp_drama_key = df.iloc[tmp_index]["drama_key"]
    if tmp_drama_key in drama_info_json.keys():
#         print(drama_info_json[tmp_drama_key]["main_actor1"])
        df["main_actor1"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor1"]
        df["main_actor2"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor2"]
#         print("yes"+tmp_drama_title)
    else:
        print(tmp_drama_key)

In [12]:
df

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key,main_actor1,main_actor2
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,...,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21,,
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,...,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22,,
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,...,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22,,
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,...,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22,,
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,...,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22,,
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,...,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22,,
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,...,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23,,
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,...,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22,,
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,...,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23,,
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,...,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22,,


In [13]:
def make_person_key_df (actor_actress):
    actor_actress_list = []
    for column in actor_actress.columns:
        if column == "ranking":
            continue
        actor_actress_list.extend(actor_actress[column].values.tolist())
    print ("元の数"+ str(len(actor_actress_list)))
    # uniqueにする
    actor_actress_list = list(set(actor_actress_list))
    #nan を除去
    actor_actress_list = [actor for actor in actor_actress_list if type(actor) is not float ]
    print ("後の数"+str(len(actor_actress_list)))

    df_pivot = pd.DataFrame( columns=actor_actress_list, index=[str(n) for n in range(2008,2020)])

    #pivot 形式に整形する
    for tmp_name in df_pivot.columns:
        for year in actor_actress.columns:
            if year == "ranking": continue
            if  len(actor_actress[actor_actress[year] == tmp_name].ranking) > 0:
                df_pivot.loc[year][tmp_name] = (actor_actress[actor_actress[year] == tmp_name].ranking).values[0]
    return (df_pivot)
    

In [14]:
df_actress = pd.read_csv("../data/actress_power.csv", encoding='cp932')
df_actress_pivot = make_person_key_df(df_actress)


元の数360
後の数62


In [15]:
df_actor = pd.read_csv("../data/actor_power.csv", encoding='cp932')
df_actor_pivot = make_person_key_df(df_actor)

元の数372
後の数65


In [16]:
df_actress_pivot

,佐々木希,仲間由紀恵,広瀬アリス,真矢みき,橋本環奈,石原さとみ,吉高由里子,木村佳乃,杏,広瀬すず,...,新垣結衣,高畑充希,松たか子,石田ゆり子,戸田恵梨香,松下奈緒,長澤まさみ,柴咲コウ,波瑠,木村文乃
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,NaN,2,NaN,18,NaN,NaN,NaN,NaN,NaN,NaN,...,9,NaN,16,NaN,25,NaN,17,7,NaN,NaN
2011,NaN,5,NaN,20,NaN,21,NaN,NaN,NaN,NaN,...,10,NaN,19,NaN,NaN,23,NaN,4,NaN,NaN
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,NaN,10,NaN,NaN,NaN,8,NaN,NaN,15,NaN,...,5,NaN,23,NaN,NaN,29,22,7,NaN,NaN
2015,21,3,NaN,NaN,NaN,12,26,NaN,7,NaN,...,5,NaN,11,NaN,NaN,NaN,22,6,NaN,NaN
2016,NaN,9,NaN,NaN,NaN,5,NaN,NaN,11,15,...,4,NaN,17,NaN,NaN,NaN,22,13,NaN,NaN
2017,30,7,NaN,NaN,NaN,3,21,NaN,14,11,...,1,NaN,NaN,20,NaN,NaN,16,NaN,NaN,NaN


In [17]:
df_actor_namekey = df_actor_pivot.T.rename_axis('actor_actress').reset_index()

In [18]:
df_actress_namekey = df_actress_pivot.T.rename_axis('actor_actress').reset_index()

In [19]:
df_actress_namekey

,actor_actress,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,佐々木希,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,NaN,30,12,16
1,仲間由紀恵,NaN,NaN,2,5,NaN,NaN,10,3,9,7,14,13
2,広瀬アリス,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,22
3,真矢みき,NaN,NaN,18,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,橋本環奈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
5,石原さとみ,NaN,NaN,NaN,21,NaN,NaN,8,12,5,3,3,5
6,吉高由里子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,NaN,21,NaN,NaN
7,木村佳乃,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30
8,杏,NaN,NaN,NaN,NaN,NaN,NaN,15,7,11,14,23,NaN
9,広瀬すず,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,11,10,7


In [37]:
df_merged = pd.merge(df, df_actor_namekey,left_on='main_actor1', right_on='actor_actress', how='left',suffixes=('_x1', '_y1'))
#df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor1', right_on='actor_actress', how='right')
#df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor2', right_on='actor_actress', how='right')

In [38]:

df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor1', right_on='actor_actress', how='left'
                     ,suffixes=('_x2', '_y2'))

In [40]:

df_merged = pd.merge(df_merged, df_actor_namekey,left_on='main_actor2', right_on='actor_actress', how='left'
                     ,suffixes=('_x3', '_y3') )


In [41]:
df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor2', right_on='actor_actress', how='left'
                    , suffixes=('_x4', '_y4'),)

In [42]:
# df_merged[["main_actor1","2017","2016","2014"]]
df_merged

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,Episode8,Episode9,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key,main_actor1,main_actor2,actor_actress_x2,2008_x2,2009_x2,2010_x2,2011_x2,2012_x2,2013_x2,2014_x2,2015_x2,2016_x2,2017_x2,2018_x2,2019_x2,actor_actress_y2,2008_y2,2009_y2,2010_y2,2011_y2,2012_y2,2013_y2,2014_y2,2015_y2,2016_y2,2017_y2,2018_y2,2019_y2,actor_actress_x4,2008_x4,2009_x4,2010_x4,2011_x4,2012_x4,2013_x4,2014_x4,2015_x4,2016_x4,2017_x4,2018_x4,2019_x4,actor_actress_y4,2008_y4,2009_y4,2010_y4,2011_y4,2012_y4,2013_y4,2014_y4,2015_y4,2016_y4,2017_y4,2018_y4,2019_y4
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,8.9,9.9,8.8,9.8,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,8.6,9,9.7,-,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,7.6,6.5,9.6,-,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,10.8,10.3,13.7,-,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,17.4,18.2,19.3,-,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,4.7,4.2,-,-,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,3.7,3.5,3.2,-,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,6.4,7,9.2,-,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,5.5,-,-,-,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,9.1,7.2,10.4,-,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_merged["actor_actress_y"]

,actor_actress_y,actor_actress_y
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [33]:
pd.set_option('display.max_columns', 500)

In [23]:
pd.set_option('display.max_rows', 1000)

In [24]:
df_merged

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,Episode8,Episode9,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key,main_actor1,main_actor2,actor_actress_x,2008_x,2009_x,2010_x,2011_x,2012_x,2013_x,2014_x,2015_x,2016_x,2017_x,2018_x,2019_x,actor_actress_y,2008_y,2009_y,2010_y,2011_y,2012_y,2013_y,2014_y,2015_y,2016_y,2017_y,2018_y,2019_y
0,TBS,土20,ブラッディ・マンデイ シーズン2,9.5,7.9,8.3,6.6,8.4,9.1,7.4,7.8,7.7,-,-,-,-,-,-,8.23%,4-4-0,2010-01-01,1001_TBS_土20,三浦春馬,佐藤健,三浦春馬,NaN,NaN,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,佐藤健,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27
1,CX,木22,白い巨塔 第二部,25.5,24.5,24.0,24.7,25.7,25.8,24.8,26,26.8,27.6,32.1,-,-,-,-,26.21%,7-3-0,2004-01-01,0401_CX_木22,唐沢寿明,江口洋介,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN,江口洋介,NaN,NaN,26,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CX,木22,白い巨塔 第一部,22.8,21.6,19.3,21.5,19.2,20.2,20.7,21.8,20.8,22.6,-,-,-,-,-,21.10%,5-4-0,2003-10-01,0310_CX_木22,唐沢寿明,江口洋介,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN,江口洋介,NaN,NaN,26,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EX,木21,BG,15.7,15.1,13.4,13.6,14,14.8,15.8,16.4,17.3,-,-,-,-,-,-,15.24%,6-2-0,2018-01-01,1801_EX_木21,木村拓哉,江口洋介,木村拓哉,NaN,NaN,4,19,NaN,NaN,27,21,21,NaN,NaN,NaN,江口洋介,NaN,NaN,26,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TBS,金22,インハンド,11.3,9.5,9.1,7.7,9,9.8,9,7.6,8.5,8.5,10.8,-,-,-,-,9.22%,2004/5/1,2019-04-01,1904_TBS_金22,山下智久,濱田岳,山下智久,NaN,NaN,13,23,NaN,NaN,NaN,NaN,NaN,NaN,28,20,濱田岳,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN
5,EX,金23,dele,5.5,3.7,4.2,3.8,4.6,5,5,4.3,-,-,-,-,-,-,-,4.51%,2003/3/1,2018-07-01,1807_EX_金23,菅田将暉,山田孝之,菅田将暉,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,16,13,山田孝之,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,8,12
6,TBS,金22,山田太郎ものがたり,17.4,15.9,16.5,13.7,15.4,11.1,13.6,14.6,16.3,17.9,-,-,-,-,-,15.34%,6-3-0,2007-07-01,0707_TBS_金22,二宮和也,櫻井翔,二宮和也,NaN,NaN,6,2,NaN,NaN,7,4,4,8,2,3,櫻井翔,NaN,NaN,1,1,NaN,NaN,3,3,2,4,3,6
7,TBS,金22,STAND UP!!,14.0,12.3,11.6,7.5,9.7,10.4,7.9,8.4,8.9,11.3,9.9,-,-,-,-,10.17%,5-5-0,2003-07-01,0307_TBS_金22,二宮和也,山下智久,二宮和也,NaN,NaN,6,2,NaN,NaN,7,4,4,8,2,3,山下智久,NaN,NaN,13,23,NaN,NaN,NaN,NaN,NaN,NaN,28,20
8,TBS,日21,MONSTERS,13.8,13.4,11.2,11.5,11,10.3,8,13.5,-,-,-,-,-,-,-,11.93%,2-5-0,2012-10-01,1210_TBS_日21,香取慎吾,山下智久,香取慎吾,NaN,NaN,2,5,NaN,NaN,9,10,15,24,18,NaN,山下智久,NaN,NaN,13,23,NaN,NaN,NaN,NaN,NaN,NaN,28,20
9,TBS,日21,GOOD LUCK!!,31.6,27.5,28.6,27.6,30.9,28.2,28.9,29.7,33.5,37.6,-,-,-,-,-,30.41%,6-3-0,2003-01-01,0301_TBS_日21,木村拓哉,堤真一,木村拓哉,NaN,NaN,4,19,NaN,NaN,27,21,21,NaN,NaN,NaN,堤真一,NaN,NaN,NaN,NaN,NaN,NaN,24,23,22,21,25,NaN
